## TRABAJO FINAL METODOLOGÍA DE LA OPINIÓN PÚBLICA

### "Análisis de resultados en las elecciones generales 2023 para CABA"


#### Grupo: Matías Chiampan, Nicolas Faccini, Camila Valeff

El objetivo del siguiente trabajo consistió en examminar los resultados electorales en la instancia de elecciones generales dentro de CABA. 
Para ello se utilizó Python y diferentes librerias importadas con el fin de robustecer la capacidad de análisis, incorporando diversidad de gráficas. 

Las librerías se importaron a partir de las funciones "import" e "install", los scripts para este paso fueron los siguientes:

In [ ]:
pip install contextily

In [ ]:
pip install geopandas

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import plotly.express as px

Luego de importar las librerias requerida, importamos los archivos .csv que son nuestra fuente de información. Tenemos dos tipos de archivos: los referidos a los resultados de las elecciones en 2023 y las bases de datos del CENSO 2010 en Argentina. Estas últimas nos permitirán contar con variables sociodemográficos para las comunas de CABA, con el fin de explorar si existe algún patrón entre éstas y las elecciones de los votantes

### Bases de datos elecciones generales 2023

In [11]:
df_resultados = pd.read_csv('2023_Generales\ResultadosElectorales_2023.csv')
df_caba = pd.read_csv('2023_Generales\CABA-circuitos-electorales.csv')
df_ambitos = pd.read_csv('2023_Generales/Ambitos_Generales_2023.csv')
df_colores = pd.read_csv('2023_Generales/Colores_2023.csv')

C:\Users\camiv\AppData\Local\Temp\ipykernel_21076\2164525572.py:1: DtypeWarning: Columns (7,10,11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resultados = pd.read_csv('2023_Generales\ResultadosElectorales_2023.csv')


#### Las tablas importadas cuentan con la siguiente estructura

In [23]:
print("Tabla resultados")
print(f"Estructura: {df_resultados.shape}")
print("\n")
df_resultados.head().T

Tabla resultados
Estructura: (5868102, 23)




,0,1,2,3,4
año,2023,2023,2023,2023,2023
eleccion_tipo,GENERAL,GENERAL,GENERAL,GENERAL,GENERAL
recuento_tipo,PROVISORIO,PROVISORIO,PROVISORIO,PROVISORIO,PROVISORIO
padron_tipo,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL
distrito_id,1,1,1,1,1
distrito_nombre,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires
seccionprovincial_id,0,0,0,0,0
seccionprovincial_nombre,NaN,NaN,NaN,NaN,NaN
seccion_id,1,1,1,1,1
seccion_nombre,Comuna 01,Comuna 01,Comuna 01,Comuna 01,Comuna 01


In [22]:
print("Circuitos electorales en CABA")
print(f"Estructura: {df_caba.shape}")
print("\n")
df_caba.head()

Circuitos electorales en CABA
Estructura: (167, 10)




,WKT,ID,CIRCUITO_N,COMUNA,BARRIO,TOTAL,MASCULINO,FEMENINO,PERIMETRO,AREA
0,MULTIPOLYGON (((-58.3661871461469 -34.62043128...,1,1,1,SAN TELMO,11377.0,5695.0,5682.0,4157.175563,7.097216e+05
1,MULTIPOLYGON (((-58.3773566900831 -34.61744732...,2,2,1,SAN TELMO,11365.0,5870.0,5495.0,3045.858735,5.234309e+05
2,MULTIPOLYGON (((-58.3572938809189 -34.59510562...,3,3,1,PUERTO MADERO,702.0,416.0,286.0,14439.585623,4.248033e+06
3,MULTIPOLYGON (((-58.3671229141027 -34.61213671...,4,4,1,PUERTO MADERO,489.0,307.0,182.0,8028.115470,7.773725e+05
4,MULTIPOLYGON (((-58.3835637305124 -34.57823507...,5,5,1,RETIRO,5275.0,3131.0,2144.0,16401.981477,3.262096e+06


In [20]:
print("Ambitos Generales")
print(f"Estructura: {df_ambitos.shape})
print("\n")
df_ambitos.head()

Ambitos Generales
(528, 7)




,año,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,seccion_id,seccion_nombre
0,2023,1,Ciudad Autónoma de Buenos Aires,0,NaN,1,Comuna 01
1,2023,1,Ciudad Autónoma de Buenos Aires,0,NaN,2,Comuna 02
2,2023,1,Ciudad Autónoma de Buenos Aires,0,NaN,3,Comuna 03
3,2023,1,Ciudad Autónoma de Buenos Aires,0,NaN,4,Comuna 04
4,2023,1,Ciudad Autónoma de Buenos Aires,0,NaN,5,Comuna 05


### Bases de datos CENSO 2010

In [7]:
Cen_viviendas = pd.read_csv('censo2010/censo/vivienda.csv')
Cen_hogares = pd.read_csv('censo2010/censo/hogar.csv')
Cen_personas = pd.read_csv('censo2010/censo/persona.csv')
Cen_dpto = pd.read_csv('censo2010/censo/dpto.csv')
Cen_frac = pd.read_csv('censo2010/censo/frac.csv')
Cen_prov = pd.read_csv('censo2010/censo/prov.csv')
Cen_radio= pd.read_csv('censo2010/censo/radio.csv')